<a href="https://colab.research.google.com/github/joiner-arrieta/Evaluaciones-de-Python-/blob/main/Evaluaci%C3%B3n_3__Manejo_de_Archivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PYTHON APLICADO A LA INGENIERIA
#**DOCENTE:** Miguel ortiz
#**ESTUDIANTE:** Joiner Arrieta
#**E-Mail:** joiner.arrieta@upb.edu.co
#**FECHA:** 20/08/2025

In [16]:
"""
================================================================================
Estudiante: Joiner Arrieta Alvarez
ID: 000483692
E-mail: joiner.arrieta@upb.edu.co
Fecha: 10-09-2025
Docente: Miguel Angel Ortiz Padilla
Actividad: Evaluación 3: Archivos, Numpy y Matplotlib
================================================================================
"""

from google.colab import drive
import numpy as np
import os

drive.mount('/content/drive')

filepath = "/content/drive/MyDrive/Colab Notebooks/GHI_Temp_Daily_Hourly_2024.csv"

print("¿Archivo encontrado?", os.path.exists(filepath))

def filter_data_by_date(filepath, year, month, day):
    """
    Filtra datos de un archivo CSV y guarda en un archivo nuevo
    filtered_data_YYYY_MM_DD.csv
    """
    try:
        data = np.genfromtxt(filepath, delimiter=";", skip_header=1)

        mask = (data[:,0] == year) & (data[:,1] == month) & (data[:,2] == day)
        filtered_data = data[mask]

        if len(filtered_data) == 0:
            print(f"No se encontraron datos para {year}-{month:02d}-{day:02d}")
            return None

        filename = f"filtered_data_{year}_{month:02d}_{day:02d}.csv"

        with open(filename, "w") as f:
            f.write("Year,Month,Day,Hour,Temperature,GHI\n")
            for row in filtered_data:
                f.write(f"{int(row[0])},{int(row[1])},{int(row[2])},{int(row[3])},{row[4]:.2f},{row[5]:.2f}\n")

        print(f"El archivo {filename} fue creado exitosamente!")
        print(f"Registros cargados: {len(filtered_data)}")

        if len(filtered_data) > 0:
            temp_avg = np.mean(filtered_data[:,4])
            ghi_max = np.max(filtered_data[:,5])
            print(f"Temperatura promedio: {temp_avg:.2f} °C")
            print(f"Radiación máxima: {ghi_max:.1f} W/m²")

        return filename

    except FileNotFoundError:
        print(f"No se encontró el archivo {filepath}")
        return None
    except IndexError:
        print("El archivo no tiene el formato esperado")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return None

def energy_calculations(filepath, year, month, day, n_panels, panel_area, panel_efficiency, inverter_efficiency, stc_temp=25, temp_coeff=-0.004):
    """
    Calcula la energía generada por un sistema FV
    """
    try:
        filtered_data_filename = filter_data_by_date(filepath, year, month, day)
        if not filtered_data_filename:
            return None

        print()

        ghi_temp_data = np.genfromtxt(filtered_data_filename, delimiter=",", skip_header=1)

        hours = ghi_temp_data[:,3]
        temp = ghi_temp_data[:,4]
        ghi = ghi_temp_data[:,5]

        total_area = panel_area * n_panels

        temp_correction = 1 + temp_coeff * (temp - stc_temp)

        dc_power = ghi * total_area * panel_efficiency * temp_correction
        ac_power = dc_power * inverter_efficiency

        hourly_energy_kwh = ac_power / 1000
        total_energy_kwh = np.sum(hourly_energy_kwh)

        energy_filename = f"energy_data_{year}_{month:02d}_{day:02d}.csv"
        with open(energy_filename, "w") as datafile:
            datafile.write("Hour,Power_DC,Power_AC,Energy_kWh\n")
            for i in range(len(hours)):
                datafile.write(f"{int(hours[i])},{dc_power[i]:.2f},{ac_power[i]:.2f},{hourly_energy_kwh[i]:.2f}\n")

        print(f"El archivo {energy_filename} fue creado exitosamente!")
        print(f"Registros cargados: {len(hours)}")
        print(f"Potencia DC promedio: {np.mean(dc_power):.2f} W")
        print(f"Potencia AC promedio: {np.mean(ac_power):.2f} W")
        print(f"Energía total generada: {total_energy_kwh:.2f} kWh")

        return energy_filename

    except Exception as e:
        print(f"Error en energy_calculations: {e}")
        return None

if __name__ == "__main__":
    print("Evaluación 3: Archivos, Numpy y Matplotlib")
    print("="*50)

    print("\n>>> PRUEBA 1: 2024-01-01")
    energy_calculations(filepath, 2024, 1, 1, 10, 1.65, 0.2, 0.95)

    print("\n>>> PRUEBA 2: 2024-02-30 (fecha inválida)")
    energy_calculations(filepath, 2024, 2, 30, 10, 1.65, 0.2, 0.95)

    print("\n>>> PRUEBA 3: 2024-12-11 (test oficial)")
    energy_calculations(filepath, 2024, 12, 11, 10, 1.65, 0.2, 0.95)

    print("\n>>> PRUEBA 4: 2024-11-15")
    energy_calculations(filepath, 2024, 11, 15, 10, 1.65, 0.2, 0.95)

    print("\n" + "="*50)
    print("ARCHIVOS CREADOS:")
    print("="*50)

    expected_files = [
        "filtered_data_2024_01_01.csv", "energy_data_2024_01_01.csv",
        "filtered_data_2024_12_11.csv", "energy_data_2024_12_11.csv",
        "filtered_data_2024_11_15.csv", "energy_data_2024_11_15.csv"
    ]

    for filename in expected_files:
        if os.path.exists(filename):
            size = os.path.getsize(filename)
            print(f"{filename} ({size} bytes)")
        else:
            print(f"{filename} - NO EXISTE")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
¿Archivo encontrado? True
Evaluación 3: Archivos, Numpy y Matplotlib

>>> PRUEBA 1: 2024-01-01
El archivo filtered_data_2024_01_01.csv fue creado exitosamente!
Registros cargados: 24
Temperatura promedio: 28.79 °C
Radiación máxima: 627.0 W/m²

El archivo energy_data_2024_01_01.csv fue creado exitosamente!
Registros cargados: 24
Potencia DC promedio: 592.44 W
Potencia AC promedio: 562.82 W
Energía total generada: 13.51 kWh

>>> PRUEBA 2: 2024-02-30 (fecha inválida)
No se encontraron datos para 2024-02-30

>>> PRUEBA 3: 2024-12-11 (test oficial)
El archivo filtered_data_2024_12_11.csv fue creado exitosamente!
Registros cargados: 24
Temperatura promedio: 28.98 °C
Radiación máxima: 728.0 W/m²

El archivo energy_data_2024_12_11.csv fue creado exitosamente!
Registros cargados: 24
Potencia DC promedio: 701.77 W
Potencia AC promedio: 666.69 W
Energía total generada: 